In [527]:
import pandas as pd

In [528]:
csv_1 = pd.read_csv('total.csv', index_col=0)

In [529]:
csv_1

,premise,hypothesis,if_else,contrastive,neutral,consensus,causal,comparative,historical,ucs_score,wucs_score,consensus_wucs_score
question,,,,,,,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,"1. The most likely answer is ""contradiction"". ...",1. Premise: A man with a white camp is standin...,"1. The commonly agreed-upon answer is ""contrad...","1. The correct answer is ""contradiction"". The ...","1. The most likely answer is ""contradiction."" ...",1. Premise: A man with a white camp is standin...,1. Premise: A man with a white camp is standin...,1. Premise: A man with a white camp is standin...,1. Premise: A man with a white camp is standin...
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,"2. The most likely answer is ""neutral"". The pr...",2. Premise: Two girls wearing skirts like the ...,"2. The commonly agreed-upon answer is ""neutral...","2. The correct answer is ""neutral"". The premis...","2. The most likely answer is ""neutral."" The pr...",2. Premise: Two girls wearing skirts like the ...,2. Premise: Two girls wearing skirts like the ...,2. Premise: Two girls wearing skirts like the ...,2. Premise: Two girls wearing skirts like the ...
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,"3. The most likely answer is ""entailment"". The...",3. Premise: Dog leaps from water while woman k...,"3. The commonly agreed-upon answer is ""entailm...","3. The correct answer is ""entailment"". The pre...","3. The most likely answer is ""entailment."" The...",3. Premise: Dog leaps from water while woman k...,3. Premise: Dog leaps from water while woman k...,3. Premise: Dog leaps from water while woman k...,3. Premise: Dog leaps from water while woman k...
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,"4. The most likely answer is ""entailment"". The...",4. Premise: A bald man in an orange shirt and ...,"4. The commonly agreed-upon answer is ""entailm...","4. The correct answer is ""entailment"". The pre...","4. The most likely answer is ""entailment."" The...",4. Premise: A bald man in an orange shirt and ...,4. Premise: A bald man in an orange shirt and ...,"4. The most likely answer is ""entailment."" The...","4. The most likely answer is ""entailment."" The..."
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,"5. The most likely answer is ""entailment"". The...",5. Premise: A boy stands beside a railing at a...,"5. The commonly agreed-upon answer is ""neutral...","5. The correct answer is ""entailment"". The pre...","5. The most likely answer is ""neutral."" The pr...",5. Premise: A boy stands beside a railing at a...,5. Premise: A boy stands beside a railing at a...,5. Premise: A boy stands beside a railing at a...,5. Premise: A boy stands beside a railing at a...
...,...,...,...,...,...,...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat a

In [456]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [457]:
prompt_template= '''
Premise: %s, 
Hypothesis: %s.
Based on what is historically known, what is the most likely answer to the premise '%s' and hypothesis %s with options "entailment", "neutral", "contradiction"? Provide historical context in your rationale.\n
'''

In [486]:
error = csv_1[csv_1.ucs_score.isna()]

In [459]:
error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis'], x['premise'], x['hypothesis']), axis=1)

In [460]:
error

,premise,hypothesis,rationales,prompt


In [461]:
print(error.prompt[:10].values)

[]


In [530]:
csv_1['split'] = csv_1.consensus_wucs_score.apply(lambda x: x.lower().split('most likely answer'))

In [531]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [532]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [535]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory', 'entail', 'contradict']:
        if choice in row['consensus_wucs_score'].lower():
            row.loc['correct_index'][choice] = row['consensus_wucs_score'].lower().index(choice)

In [536]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [537]:
csv_1[csv_1.LLM_answer.isna()]

,premise,hypothesis,if_else,contrastive,neutral,consensus,causal,comparative,historical,ucs_score,wucs_score,consensus_wucs_score,split,correct_index,LLM_answer
question,,,,,,,,,,,,,,,


In [481]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [538]:
paper = pd.read_csv('../paper.csv', index_col=0)

In [539]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']
csv_1.set_index('question', inplace=True)

In [540]:
paper[~paper.premise.isin(csv_1.premise)]['hypothesis'].values

array([], dtype=object)

In [541]:
paper.shape

(5592, 6)

In [542]:
total_df = csv_1[(csv_1.premise.isin(paper.premise)) & (csv_1.hypothesis.isin(paper.hypothesis))].drop_duplicates(subset=['premise', 'hypothesis'])
total_df

,premise,hypothesis,if_else,contrastive,neutral,consensus,causal,comparative,historical,ucs_score,wucs_score,consensus_wucs_score,split,correct_index,LLM_answer
question,,,,,,,,,,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,"1. The most likely answer is ""contradiction"". ...",1. Premise: A man with a white camp is standin...,"1. The commonly agreed-upon answer is ""contrad...","1. The correct answer is ""contradiction"". The ...","1. The most likely answer is ""contradiction."" ...",1. Premise: A man with a white camp is standin...,1. Premise: A man with a white camp is standin...,1. Premise: A man with a white camp is standin...,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,{'contradiction': 194},contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,"2. The most likely answer is ""neutral"". The pr...",2. Premise: Two girls wearing skirts like the ...,"2. The commonly agreed-upon answer is ""neutral...","2. The correct answer is ""neutral"". The premis...","2. The most likely answer is ""neutral."" The pr...",2. Premise: Two girls wearing skirts like the ...,2. Premise: Two girls wearing skirts like the ...,2. Premise: Two girls wearing skirts like the ...,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'neutral': 119},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,"3. The most likely answer is ""entailment"". The...",3. Premise: Dog leaps from water while woman k...,"3. The commonly agreed-upon answer is ""entailm...","3. The correct answer is ""entailment"". The pre...","3. The most likely answer is ""entailment."" The...",3. Premise: Dog leaps from water while woman k...,3. Premise: Dog leaps from water while woman k...,3. Premise: Dog leaps from water while woman k...,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'entailment': 143},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,"4. The most likely answer is ""entailment"". The...",4. Premise: A bald man in an orange shirt and ...,"4. The commonly agreed-upon answer is ""entailm...","4. The correct answer is ""entailment"". The pre...","4. The most likely answer is ""entailment."" The...",4. Premise: A bald man in an orange shirt and ...,4. Premise: A bald man in an orange shirt and ...,"4. The most likely answer is ""entailment."" The...","4. The most likely answer is ""entailment."" The...","is ""entailment."" the premise describes a bald...",{'entailment': 5},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,"5. The most likely answer is ""entailment"". The...",5. Premise: A boy stands beside a railing at a...,"5. The commonly agreed-upon answer is ""neutral...","5. The correct answer is ""entailment"". The pre...","5. The most likely answer is ""neutral."" The pr...",5. Premise: A boy stands beside a railing at a...,5. Premise: A boy st

In [543]:
total_df.loc[total_df.LLM_answer == 'entailed', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'entails', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradicts', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradicted', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradictory', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'entail', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradict', 'LLM_answer'] = 'contradiction'

In [544]:
total_df

,premise,hypothesis,if_else,contrastive,neutral,consensus,causal,comparative,historical,ucs_score,wucs_score,consensus_wucs_score,split,correct_index,LLM_answer
question,,,,,,,,,,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,"1. The most likely answer is ""contradiction"". ...",1. Premise: A man with a white camp is standin...,"1. The commonly agreed-upon answer is ""contrad...","1. The correct answer is ""contradiction"". The ...","1. The most likely answer is ""contradiction."" ...",1. Premise: A man with a white camp is standin...,1. Premise: A man with a white camp is standin...,1. Premise: A man with a white camp is standin...,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,{'contradiction': 194},contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,"2. The most likely answer is ""neutral"". The pr...",2. Premise: Two girls wearing skirts like the ...,"2. The commonly agreed-upon answer is ""neutral...","2. The correct answer is ""neutral"". The premis...","2. The most likely answer is ""neutral."" The pr...",2. Premise: Two girls wearing skirts like the ...,2. Premise: Two girls wearing skirts like the ...,2. Premise: Two girls wearing skirts like the ...,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'neutral': 119},neutral
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,"3. The most likely answer is ""entailment"". The...",3. Premise: Dog leaps from water while woman k...,"3. The commonly agreed-upon answer is ""entailm...","3. The correct answer is ""entailment"". The pre...","3. The most likely answer is ""entailment."" The...",3. Premise: Dog leaps from water while woman k...,3. Premise: Dog leaps from water while woman k...,3. Premise: Dog leaps from water while woman k...,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'entailment': 143},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,"4. The most likely answer is ""entailment"". The...",4. Premise: A bald man in an orange shirt and ...,"4. The commonly agreed-upon answer is ""entailm...","4. The correct answer is ""entailment"". The pre...","4. The most likely answer is ""entailment."" The...",4. Premise: A bald man in an orange shirt and ...,4. Premise: A bald man in an orange shirt and ...,"4. The most likely answer is ""entailment."" The...","4. The most likely answer is ""entailment."" The...","is ""entailment."" the premise describes a bald...",{'entailment': 5},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,"5. The most likely answer is ""entailment"". The...",5. Premise: A boy stands beside a railing at a...,"5. The commonly agreed-upon answer is ""neutral...","5. The correct answer is ""entailment"". The pre...","5. The most likely answer is ""neutral."" The pr...",5. Premise: A boy stands beside a railing at a...,5. Premise: A boy st

In [545]:
paper['my_label'] = total_df['LLM_answer']

In [546]:
sum(paper.label == paper.my_label) / len(paper)

0.6348354792560801

In [547]:
sum(paper.label == paper.llm_label) / len(paper)

0.699928469241774

In [526]:
total_df.to_csv('consensus_wucs_score - full.csv')

In [195]:
csv_1.to_csv('neutral - full.csv', index=False)

In [162]:
total_df.index.value_counts()

A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.                                                                     1
a man taking a pizza out of an oven</s>A woman is sitting in the oven.                                                                                                                                                                  1
Side view of a mountaineer walking into strong headwinds.</s>A man outside.                                                                                                                                                             1
A boy jumping off of a diving board while a girl watches.</s>A boy is near water.                                                                                                                                                       1
A back view of a man driving a tractor and signaling with two ch